In [1]:
!pip install rnnmorph

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 7.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 45.5 MB/s eta 0:00:00
  Created wheel for rnnmorph: filename=rnnmorph-0.4.1-py3-none-any.whl size=19746378 sha256=e15d49b909f182e245e4a36bb42c3cf5633984e8c74ea1aec70437842a1d0505
  Stored in directory: /root/.cache/pip/wheels/4b/47/48/13dc27987d34abf46ca5fb2877fb993edd01cd95153863f84d
  Created wheel for russian-tagsets: filename=russian_tagsets-0.6-py3-none-any.whl size=24637 sha256=8312d923892bc3b4a81041b422871916156d1f548511d35e58233ef36e087e30
  Stored in directory: /root/

In [2]:
from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")
forms = predictor.predict(["мама", "мыла", "раму"])

1/1 [==============================] - 5s 5s/step


In [7]:
DATA_URL = "http://az.lib.ru/l/leskow_n_s/text_0246.shtml" # Ссылка на файл из Задания
more_words_count = 100
tokens_more_count = 50

In [8]:
import nltk
import warnings
import urllib.request
from tqdm import tqdm
from nltk import FreqDist
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize, word_tokenize
warnings.filterwarnings('ignore')
nltk.download('punkt', quiet=True)
nltk.download("stopwords", quiet=True)

from rnnmorph.predictor import RNNMorphPredictor
predictor = RNNMorphPredictor(language="ru")
opener = urllib.request.URLopener({})
resource = opener.open(DATA_URL)
raw_text = resource.read().decode(resource.headers.get_content_charset())

soup = BeautifulSoup(raw_text, features="html.parser")

for script in soup(["script", "style"]):
    script.extract()

cleaned_text = soup.get_text()
tokenized_sentences = [word_tokenize(sentence) for sentence in sent_tokenize(cleaned_text)]
predictions = [[pred.normal_form for pred in sent if pred.normal_form.isalpha()] for sent in tqdm(predictor.predict_sentences(sentences=tokenized_sentences), "sentences") ] # ИСПРАВИЛ НА ТОЛЬКО БУКВЫ
non_uniq_tokens = [word for sentence in predictions for word in sentence]

answers = {}
for i in non_uniq_tokens:
  if i in answers:
    answers[i] += 1
  else:
    answers[i] = 1

STOPWORDS = set(stopwords.words("russian"))

t = 1
counter = more_words_count + 1
while counter > more_words_count:
  counter = 0
  n_answers = []
  for i in answers:
    if (answers[i] > t):
      n_answers.append(i)
      counter += 1
  t += 1

counter = 0
for i in n_answers:
  if i in stopwords.words("russian"):
    counter += 1

counter1 = 0
for i in answers:
  if (answers[i] > tokens_more_count):
    counter1 += 1
print("\n")

print("Введите количество предложений")
print("Ответ:", len(predictions))

print("Введите количество токенов, состоящих только из букв")
print("Ответ:", len(non_uniq_tokens))

print(f"Какую долю среди {more_words_count} самых частотных токенов в произведении занимают слова, не входящие в стоп-лист?")
print("Ответ:", (more_words_count - counter) / more_words_count)

print(f"Сколько токенов встречается в тексте строго больше {tokens_more_count} раз?")
print("Ответ:", counter1)

8/8 [==============================] - 6s 563ms/step


sentences: 100%|██████████| 466/466 [00:00<00:00, 62104.27it/s]



Введите количество предложений
Ответ: 466
Введите количество токенов, состоящих только из букв
Ответ: 9053
Какую долю среди 100 самых частотных токенов в произведении занимают слова, не входящие в стоп-лист?
Ответ: 0.46
Сколько токенов встречается в тексте строго больше 50 раз?
Ответ: 22


In [ ]:
print(forms[0].tag)

Case=Nom|Gender=Fem|Number=Sing


In [ ]:
print(forms[0].normal_form)

мама


In [ ]:
print(forms[0].vector)

[0 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 1 0 0 1 0 1 0 0 0 1 0 0 1]
